In [1]:
# === Setup & Configuration ===
__version__ = "1.0.0"
print(f"Trend Portfolio Analysis v{__version__}")

import pandas as pd
import numpy as np
import yaml
import os
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import DateOffset
from google.colab import files

warnings.filterwarnings("ignore")

# 1) Upload & read CSV
print("✓ Uploading CSV file…")
uploaded = files.upload()
csv_path = next(iter(uploaded))
print(f"✓ Loaded CSV: {csv_path}")

# Robust file reading: handle CSV or Excel
import pandas as pd
import os
file_ext = os.path.splitext(csv_path)[1].lower()
if file_ext in ('.xls', '.xlsx', '.xlsm'):
    raw_df = pd.read_excel(csv_path, index_col=0)
else:
    try:
        raw_df = pd.read_csv(csv_path, index_col=0, na_values=[''])
    except pd.errors.ParserError:
        # fallback: infer delimiter
        raw_df = pd.read_csv(csv_path, index_col=0, sep=None, engine='python')
raw_df.index = pd.to_datetime(raw_df.index).to_period('M').to_timestamp('M')
raw_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
raw_df.sort_index(inplace=True)

# 2) Auto-truncate post-exit
print("✓ Auto-detecting fund lifetimes and truncating post-exit data…")
fund_cols = [
    c for c in raw_df.columns
    if c not in ("Risk_Free", "S&P 500", "Bloomberg Barclays Agg", "60/40 Portfolio")
       and not c.startswith("EqualWeight_")
]
for fund in fund_cols:
    s = raw_df[fund]
    if not s.dropna().empty:
        exit_dt = s.last_valid_index()
        raw_df.loc[raw_df.index > exit_dt, fund] = np.nan

# 3) Split out series
print("✓ Separating risk-free rate and fund returns…")
rf_series    = raw_df.iloc[:, 0]
fund_returns = raw_df[fund_cols]

# 4) Load YAML config
print("✓ Loading configuration…")
config = {}
cfg_path = "trend_portfolio_config.yaml"
if os.path.exists(cfg_path):
    with open(cfg_path) as f:
        config = yaml.safe_load(f) or {}
print("Configuration:", config)

Trend Portfolio Analysis v1.0.0
✓ Uploading CSV file…


Saving hedge_fund_returns_with_indexes.csv to hedge_fund_returns_with_indexes.csv
✓ Loaded CSV: hedge_fund_returns_with_indexes.csv
✓ Auto-detecting fund lifetimes and truncating post-exit data…
✓ Separating risk-free rate and fund returns…
✓ Loading configuration…
Configuration: {}


In [2]:
# === Utility Functions ===
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset

def parse_ym(ym_str):
    """Parse 'YYYY-MM' into a month-end Timestamp."""
    return pd.to_datetime(ym_str + '-01').to_period('M').to_timestamp('M')

def select_funds_with_complete_data(returns, start_date, end_date):
    """Return funds with complete in-sample and out-of-sample data."""
    from pandas.tseries.offsets import DateOffset

    out_start = end_date + DateOffset(months=1)
    out_end   = out_start + DateOffset(months=11)

    # Exclude any risk-free column
    funds = [
        f for f in returns.columns
        if f.lower() not in ("risk_free", "rf", "risk free")
    ]

    valid = []
    for f in funds:
        s = returns[f]
        if (
            s.loc[start_date:end_date].notna().all()
            and s.loc[out_start:out_end].notna().all()
        ):
            valid.append(f)

    print(f"✓ Filtered {len(valid)}/{len(funds)} eligible funds")
    return valid

def adjust_returns(raw_returns, target_vol, leverage_cost):
    """Scale returns to target volatility, subtract cost."""
    ann_vol = raw_returns.std() * np.sqrt(12)
    factor = target_vol / ann_vol
    factor = factor.replace([np.inf, -np.inf], 0).fillna(0)
    return raw_returns.multiply(factor, axis=1).sub(leverage_cost)

def compute_summary_stats(df, rf):
    """Compute annualized return, volatility, and Sharpe ratio per fund."""
    # Annualized return per fund
    ann_return      = ((1 + df).prod() ** (12 / len(df))) - 1
    # Annualized volatility per fund
    ann_volatility  = df.std() * np.sqrt(12)
    # Excess return per fund
    excess_return   = ann_return - (rf.mean() * 12)
    # Sharpe ratio per fund
    sharpe          = excess_return / ann_volatility
    # Build DataFrame: one row per fund
    return pd.DataFrame({
        "Ann Return":      ann_return * 100,
        "Ann Volatility":  ann_volatility * 100,
        "Sharpe Ratio":    sharpe
    })
def compute_portfolio_stats(df, rf, weights):
    """Compute portfolio stats given weights."""
    # Robustness check: make sure Risk_Free isn't in df.columns
    if any(col.lower() in ("risk_free","rf","risk free") for col in df.columns):
        raise ValueError("Risk-free rate must not be included among portfolio funds")
    port = df.dot(weights)

In [6]:
def export_to_excel_bt_os(stats_bt, stats_os,
                          port_bt, port_os,
                          path,
                          start_date, end_date,
                          out_start, out_end,
                          selected_funds):
    """
    Export in-sample (BT) and out-of-sample (OS) stats and portfolio stats
    to a styled Excel file, then validate row counts.
    """

    # --- Prepare DataFrames with Fund column ---
    df_bt = stats_bt.copy().reset_index().rename(columns={'index': 'Fund'})
    df_os = stats_os.copy().reset_index().rename(columns={'index': 'Fund'})

    # Portfolio rows
    port_bt_df = pd.DataFrame([port_bt]).reset_index().rename(columns={'index': 'Fund'})
    port_os_df = pd.DataFrame([port_os]).reset_index().rename(columns={'index': 'Fund'})
    port_bt_df['Fund'] = 'Equal Weight Portfolio'
    port_os_df['Fund'] = 'Equal Weight Portfolio'

    # Concat funds + portfolio
    df_bt = pd.concat([df_bt, port_bt_df], ignore_index=True)
    df_os = pd.concat([df_os, port_os_df], ignore_index=True)

    # Convert percentage columns back to decimals for Excel formatting
    df_bt['Ann Return']     = (df_bt['Ann Return'] / 100).round(3)
    df_bt['Ann Volatility'] = (df_bt['Ann Volatility'] / 100).round(3)
    df_bt['Sharpe Ratio']   = df_bt['Sharpe Ratio'].round(2)

    df_os['Ann Return']     = (df_os['Ann Return'] / 100).round(3)
    df_os['Ann Volatility'] = (df_os['Ann Volatility'] / 100).round(3)
    df_os['Sharpe Ratio']   = df_os['Sharpe Ratio'].round(2)

    # --- Write to Excel with styling ---
    with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
        workbook = writer.book

        # Formats
        title_fmt  = workbook.add_format({'bold': True, 'font_size': 14, 'align': 'center'})
        header_fmt = workbook.add_format({'bold': True, 'bg_color': '#D7E4BC',
                                          'border': 1, 'align': 'center'})
        pct_fmt    = workbook.add_format({'num_format': '0.0%', 'border': 1})
        dec2_fmt   = workbook.add_format({'num_format': '0.00', 'border': 1})

        def write_sheet(df, sheet_name, title_text):
            ws = workbook.add_worksheet(sheet_name)
            writer.sheets[sheet_name] = ws

            # Merged title row A1:D1
            ws.merge_range('A1:D1', title_text, title_fmt)

            # Write the DataFrame starting at row 3 (zero-indexed row 2)
            df.to_excel(writer, sheet_name=sheet_name, startrow=2, index=False)

            # Header row formatting
            for col_idx, col in enumerate(df.columns):
                ws.write(2, col_idx, col, header_fmt)

            # Column widths & formats
            ws.set_column('A:A', 30)       # Fund
            ws.set_column('B:B', 15, pct_fmt)  # Ann Return
            ws.set_column('C:C', 18, pct_fmt)  # Ann Volatility
            ws.set_column('D:D', 15, dec2_fmt) # Sharpe Ratio

            # Freeze panes below header
            ws.freeze_panes(3, 0)

            # Autofilter on header row
            ws.autofilter(2, 0, 2 + len(df), len(df.columns) - 1)

        # In-Sample sheet
        bt_name = f"{start_date.strftime('%y-%m')}-{end_date.strftime('%y-%m')} BT"
        bt_title = f"In-Sample Results ({start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')})"
        write_sheet(df_bt, bt_name, bt_title)

        # Out-of-Sample sheet
        os_name = f"{out_start.strftime('%y-%m')}-{out_end.strftime('%y-%m')} OS"
        os_title = f"Out-of-Sample Results ({out_start.strftime('%Y-%m')} to {out_end.strftime('%Y-%m')})"
        write_sheet(df_os, os_name, os_title)

    print(f"✅ Exported styled Excel results to {path}")



In [7]:
# === Validate & Input Parameters ===
from pandas.tseries.offsets import DateOffset

# Helper to pull from config or prompt
def get_param(key, prompt):
    val = config.get(key, "")
    while not val:
        val = input(f"{prompt}").strip()
    return val

# Dates
start_date = parse_ym(get_param("start_date", "Enter start date (YYYY-MM): "))
end_date   = parse_ym(get_param("end_date",   "Enter end date (YYYY-MM): "))
if start_date >= end_date:
    raise ValueError("start_date must be before end_date")

# Target volatility
target_vol = float(get_param("target_volatility", "Target volatility (e.g. 0.10): "))
if target_vol <= 0:
    raise ValueError("target_volatility must be > 0")

# Leverage cost
leverage_cost = float(get_param("leverage_cost", "Leverage cost (e.g. 0.0025): "))
if leverage_cost < 0:
    raise ValueError("leverage_cost must be >= 0")

# Selection mode
selection_mode = get_param("selection_mode", "Selection mode [all/random/manual]: ").lower()
if selection_mode not in ("all", "random", "manual"):
    raise ValueError("selection_mode must be one of all, random, manual")
print(f"✓ Selection mode: {selection_mode}")

# Sample size
sample_size = int(get_param("sample_size", "Sample size (if random): "))
if sample_size <= 0:
    raise ValueError("sample_size must be > 0")

# Out-of-sample window check
out_start = end_date + DateOffset(months=1)
out_end   = out_start   + DateOffset(months=11)
last_date = fund_returns.index.max()
if out_end > last_date:
    raise ValueError(f"OOS end {out_end.date()} exceeds data max {last_date.date()}")
print("✓ Parameter validation complete")


Enter start date (YYYY-MM): 2005-07
Enter end date (YYYY-MM): 2008-06
Target volatility (e.g. 0.10): .25
Leverage cost (e.g. 0.0025): .003
Selection mode [all/random/manual]: random
✓ Selection mode: random
Sample size (if random): 15
✓ Parameter validation complete


In [8]:
# === Main Execution ===
# Select funds
if selection_mode == "all":
    selected_funds = select_funds_with_complete_data(fund_returns, start_date, end_date)
elif selection_mode == "random":
    eligible = select_funds_with_complete_data(fund_returns, start_date, end_date)
    selected_funds = list(np.random.choice(eligible, size=min(sample_size,len(eligible)), replace=False))
    print(f"✓ Randomly selected {len(selected_funds)} funds")
else:  # manual
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    checkboxes = [widgets.Checkbox(f, False) for f in fund_returns.columns]
    button = widgets.Button(description="Confirm Selection")
    out = widgets.Output()
    def confirm(b):
        with out:
            clear_output()
            global selected_funds
            selected_funds = [cb.description for cb in checkboxes if cb.value]
            if not selected_funds:
                print("Error: At least one fund must be selected")
            else:
                print(f"✓ Manual selection: {len(selected_funds)} funds")
    button.on_click(confirm)
    display(widgets.VBox(checkboxes + [button, out]))

# Wait for manual selection if needed
    time.sleep(1)

# Compute returns slices
returns_bt = fund_returns[selected_funds].loc[start_date:end_date]
returns_os = fund_returns[selected_funds].loc[out_start:out_end]

# Adjust returns
adj_bt = adjust_returns(returns_bt, target_vol, leverage_cost)
adj_os = adjust_returns(returns_os, target_vol, leverage_cost)
print("✓ Returns adjusted")

# Compute stats
stats_bt = compute_summary_stats(adj_bt, rf_series.loc[start_date:end_date])
stats_os = compute_summary_stats(adj_os, rf_series.loc[out_start:out_end])
print("✓ Summary statistics computed")

# Portfolio stats
weights = np.repeat(1/len(selected_funds), len(selected_funds))
port_bt = compute_portfolio_stats(adj_bt, rf_series.loc[start_date:end_date], weights)
port_os = compute_portfolio_stats(adj_os, rf_series.loc[out_start:out_end], weights)
print("✓ Portfolio statistics computed")

✓ Filtered 28/101 eligible funds
✓ Randomly selected 15 funds
✓ Returns adjusted
✓ Summary statistics computed
✓ Portfolio statistics computed


In [13]:
!pip install xlsxwriter
# === Export Results ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_file = f"Trend_Portfolio_Results_v{__version__}_{timestamp}.xlsx"
export_to_excel_bt_os(stats_bt, stats_os, port_bt, port_os, output_file, start_date, end_date, out_start, out_end, selected_funds)



✅ Exported styled Excel results to Trend_Portfolio_Results_v1.0.0_20250530_0043.xlsx
